In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# for jupyter notebook
pd.options.mode.chained_assignment = None  # default='warn'

# Search

Search for right ITEMID with regex

In [ ]:
# read in data to pandas
df = pd.read_csv('mimic-iii-clinical-database-1.4/D_ITEMS.csv.gz')

In [ ]:
df['LABEL'] = df['LABEL'].str.lower()

df[df['LABEL'].str.contains("_*weight*", na=False)]

List up labels for certain ITEMID

In [ ]:
df[df['ITEMID'].isin([920, 1394, 4187, 3486, 3485, 4188, 226707, 226730])]

# ITEMID

My choosen ITEMID, some from mimic GitHub page other found with regex

In [ ]:
df = pd.read_csv('mimic-iii-clinical-database-1.4/CHARTEVENTS.csv.gz')

df

In [ ]:
## Take around 10-13 min, select items in check and remove those with error

def makedata(name, check, minVal, maxVal):
    df = pd.DataFrame()
    chunksize = 10**7

    for chunk in pd.read_csv('mimic-iii-clinical-database-1.4/CHARTEVENTS.csv.gz', chunksize=chunksize):
        chunk = chunk[ (chunk['ITEMID'].isin(check)) & (chunk['ERROR'] != 1) & (chunk['VALUENUM'] > minVal) & (chunk['VALUENUM'] < maxVal)] 
        df = df.append(chunk)
    
    df.set_index('ROW_ID')
    df.to_csv(name, encoding="utf-8")

In [ ]:
## REGEX

## -- Temperature
## ITEMID       LABEL
## 677          temperature c (calc) -- why nt used

## -- Oxygen
## ITEMID      LABEL
## 7146        oxygen #3
## 228231      par-oxygen sautratuion

#check = [7146, 228231]

## -- Surgery


## -- Mental status

In [ ]:
# source mimic github hieght and weight
# https://github.com/MIT-LCP/mimic-code/blob/master/concepts/demographics/HeightWeightQuery.sql

## -- Height                      --> time: 10 min
# ITEMID        LABEL
# 920           Admit Ht
# 4187          Length Calc Inches
# 4188          Length in cm
# 1394          Height Inches
# 3485          Length Calc (cm)
# 3486          Length in Inches
# 226707        Height

## -- Weight
## ITEMID       LABEL
## 762          daily weight
## 763          admit wt
## 3723         bith weight
## 3580         weight kg
## 3581         weigth lb
## 3582         weight oz

#762, 763, 3723, 3580,                     -- Weight Kg
#3581,                                     -- Weight lb
#3582,                                     -- Weight oz
#920, 1394, 4187, 3486,                    -- Height inches
#3485, 4188                                -- Height cm
#-- Metavision
#    , 226707 -- Height (measured in inches)
#    , 226512 -- Admission Weight (Kg)

#    -- note we intentionally ignore the below ITEMIDs in metavision
#    -- these are duplicate data in a different unit
#    -- , 226531 -- Admission Weight (lbs.)
#    -- , 226730 -- Height (cm)

In [ ]:
## Source mimic code GitHub
## https://github.com/MIT-LCP/mimic-code/blob/master/concepts/firstday/vitals-first-day.sql

##  -- HEART RATE
##  211, --"Heart Rate"
##  220045, --"Heart Rate"

##  -- Systolic/diastolic

##  51, --	Arterial BP [Systolic]
##  442, --	Manual BP [Systolic]
##  455, --	NBP [Systolic]
##  6701, --	Arterial BP #2 [Systolic]
##  220179, --	Non Invasive Blood Pressure systolic
##  220050, --	Arterial Blood Pressure systolic

##  8368, --	Arterial BP [Diastolic]
##  8440, --	Manual BP [Diastolic]
##  8441, --	NBP [Diastolic]
##  8555, --	Arterial BP #2 [Diastolic]
##  220180, --	Non Invasive Blood Pressure diastolic
##  220051, --	Arterial Blood Pressure diastolic


##  -- MEAN ARTERIAL PRESSURE
##  456, --"NBP Mean"
##  52, --"Arterial BP Mean"
##  6702, --	Arterial BP Mean #2
##  443, --	Manual BP Mean(calc)
##  220052, --"Arterial Blood Pressure mean"
##  220181, --"Non Invasive Blood Pressure mean"
##  225312, --"ART BP mean"

##  -- RESPIRATORY RATE
##  618,--	Respiratory Rate
##  615,--	Resp Rate (Total)
##  220210,--	Respiratory Rate
##  224690, --	Respiratory Rate (Total)


##  -- SPO2, peripheral
##  646, 220277,

##  -- GLUCOSE, both lab and fingerstick
##  807,--	Fingerstick Glucose
##  811,--	Glucose (70-105)
##  1529,--	Glucose
##  3745,--	BloodGlucose
##  3744,--	Blood Glucose
##  225664,--	Glucose finger stick
##  220621,--	Glucose (serum)
##  226537,--	Glucose (whole blood)

##  -- TEMPERATURE
##  223762, -- "Temperature Celsius"
##  676,	-- "Temperature C"
##  223761, -- "Temperature Fahrenheit"
##  678 --	"Temperature F"

# Make data

In [ ]:
%%time
makedata("mimic-iii-clinical-database-1.4/vital_Height.csv", [920, 1394, 4187, 3486, 3485, 4188, 226707], 0, 10000)

In [ ]:
%%time
makedata("mimic-iii-clinical-database-1.4/vital_Weight.csv", [762, 763, 3723, 3580, 3581, 3582, 226512], 0, 10000)

In [ ]:
%%time
makedata("mimic-iii-clinical-database-1.4/vital_HeartRate.csv", [211,220045], 0, 300)

In [ ]:
%%time
makedata("~/vital_sysBP.csv", [51,442,455,6701,220179,220050], 0, 400)

In [ ]:
%%time
makedata("~/vital_diasBP.csv", [8368,8440,8441,8555,220180,220051], 0, 300)

In [ ]:
%%time
makedata("~/vital_respRate.csv", [615,618,220210,224690], 0, 70)

In [ ]:
%%time
makedata("~/vital_temp_F.csv", [223761,678], 70, 120)

In [ ]:
%%time
makedata("~/vital_temp_C.csv", [223762,676], 10, 50)

In [ ]:
%%time
makedata("~/vital_oxygen_met.csv", [646,220277], 0, 101)

In [ ]:
%%time
makedata("~/vital_gluc.csv", [807,811,1529,3745,3744,225664,220621,226537], 0, 1000000)

Save output data since the making take long time on pc with low RAM

# Convert data

In [ ]:
# method for convertion
def convert(source, convertable, val):
    source.loc[source['ITEMID'].isin(convertable), 'VALUE'] *= val
    source.loc[source['ITEMID'].isin(convertable), 'VALUENUM'] *= val


### make tables with average in 6 hours

In [ ]:
def makeprocessdata(name_val, df, hw):
    # groupby
    gb = df.groupby("ICUSTAY_ID")
    
    # iterate over the groups
    df_new = pd.DataFrame(columns=['ICUSTAY_ID', name_val])
    i = 0
     

    # name is the gruop name, and group is the dataframe
    for name, group in gb:
        if hw:
            group['CHARTTIME'] = pd.to_datetime(group['CHARTTIME'])
            group['INTIME'] = pd.to_datetime(group['INTIME'])

            # take the difference, to calculate hours from intime
            group['DIFF'] = (group['CHARTTIME'] - group['INTIME']).astype('timedelta64[h]')

            # only takes whose with value between 6 hours
            group = group[(group['DIFF'] >= -6.0) & (group['DIFF'] <= 6.0)]

        # intialise data of lists. 
        df_new.loc[i] = [name, group['VALUENUM'].mean()]
        i += 1
        #data = {'ICUSTAY_ID':[name], name_val:group['VALUENUM'].mean()} 

        # add to DataFrame
        #df_new = df_new.append(pd.DataFrame(data))
    
    print("SAVING...")
    #df_new.set_index('ICUSTAY_ID')
    df_new.to_csv("~/ICU_"+name_val+".csv", encoding="utf-8", index=False)

# Filter data in hours and take mean

In [ ]:
def processdata(name, height, weight, temp_f):
    print("------ "+name+" ------")
    df = pd.read_csv("~/vital_"+name+".csv", encoding="utf-8")
    
    df = df[['ICUSTAY_ID', 'ITEMID', 'CHARTTIME', 'VALUE', 'VALUENUM']]
    
    if height:
        print("YES: HEIGTH Converting...")
        # Ensure that all heights are in centimeters
        convertable = [920, 1394, 4187, 3486, 226707]
        convert(df, convertable, 2.54)
    
    
    if weight:
        print("YES: WEIGTH Converting...")
        # Ensure that all weights are in kg and heights are in centimeters
        convertable = [3581, 226531]
        convert(df, convertable, 0.45359237)

        convertable = [3582]
        convert(df, convertable, 0.0283495231)
        
    if temp_f:
        print("YES: TEMP_F Converting...")
        # convert F to C
        # case when itemid in (223761,678) then (valuenum-32)/1.8 else valuenum end as valuenum

        df.loc[df['ITEMID'].isin([223761,678]), 'VALUE'] = (df['VALUE'] - 32)/1.8
        df.loc[df['ITEMID'].isin([223761,678]), 'VALUENUM'] = (df['VALUENUM'] - 32)/1.8

    
    # merge with icustays to get icustay id
    df_los = pd.read_csv("~/mimic-iii-clinical-database-1.4/ICUSTAYS.csv.gz")
    df = pd.merge(df, df_los, on='ICUSTAY_ID', how='left')
    
    if height or weight:
        print("MAKE: HEIGTH/WEIGHT")
        makeprocessdata(name, df, False)
    else:
        print("MAKE: ALL")
        makeprocessdata(name, df, True)    

# Read in and process data

In [ ]:
df = processdata("Height", True, False, False)
df = processdata("Weight", False, True, False)

In [ ]:
%%time
## around 40 min

df = processdata("HeartRate", False, False, False)
df = processdata("sysBP", False, False, False)
df = processdata("diasBP", False, False, False)
df = processdata("respRate", False, False, False)
#df = processdata("temp_F", False, False, True)
#df = processdata("temp_C", False, False, False)
df = processdata("oxygen_met", False, False, False)
df = processdata("gluc", False, False, False)

In [ ]:
df = pd.read_csv("/home/sohaib/ICU_Height.csv", encoding="utf-8")

# Make BMI

In [ ]:
# read in weight and height
df_height = pd.read_csv("~/ICU_Height.csv", encoding="utf-8")
df_weight = pd.read_csv("~/ICU_Weight.csv", encoding="utf-8")

In [ ]:
df_bmi = pd.merge(df_weight, df_height, on='ICUSTAY_ID')

In [ ]:
# calculate BMI
def bmi(height, weight):
    return (weight / (height*height))

In [ ]:
df_bmi['BMI'] = bmi(df_bmi['Weight'], df_bmi['Height'])

In [ ]:
df_bmi = df_bmi[['ICUSTAY_ID', 'BMI']]

In [ ]:
df_bmi.set_index('ICUSTAY_ID')
df_bmi.to_csv("~/ICU_BMI.csv", encoding="utf-8")

# merge temp dataframes

In [ ]:
# read in weight and height
df_f = pd.read_csv("~/ICU_temp_F.csv", encoding="utf-8")
df_f = df_f.rename(index=str, columns={"temp_F": "temp_C"})
df_c = pd.read_csv("~/ICU_temp_C.csv", encoding="utf-8")

In [ ]:
df_temp = df_c.append(df_f)

df_temp

In [ ]:
#df_temp.set_index('ICUSTAY_ID')
df_temp.to_csv("~/ICU_temp.csv", encoding="utf-8")

df = processdata("", True, False, False)

# plot singel vital for one patient

In [ ]:
df2['CHARTTIME'] = pd.to_datetime(df2['CHARTTIME'])

df2[['VALUE','CHARTTIME']].set_index('CHARTTIME').plot()

df = pd.merge(df, df2, on="ITEMID")

df2 = df2[df2['ICUSTAY_ID'] == 200019]

df2

df2['CHARTTIME'] = pd.to_datetime(df2['CHARTTIME'])

df2[['VALUE','CHARTTIME']].set_index('CHARTTIME').plot()